In [1]:
%cd ..

/home/ltorres/leo/Cloud-Classification/code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(7)
np.random.seed(7)

In [5]:
import random

In [6]:
import os
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import glob

In [7]:
from src import config
from src import engine
from src import utils

In [8]:
from recipe.utils import *

In [9]:
torch.manual_seed(7)
np.random.seed(7)

### Dataloaders

In [10]:
def get_gcd_paths(data_dir, dataset_type):
    return glob.glob(
                     os.path.join(data_dir,f'GCD/{dataset_type}/**/*.jpg'), 
                     recursive=True
                    )

In [11]:
path_train_images = get_gcd_paths('/data/mandonaire','train')
#random.shuffle(path_train_images)


train_dataset = GCD(path_train_images, resize=224)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=8,
        num_workers=8,
        shuffle=True,
    )

In [12]:
len(train_dataset)

10000

In [13]:
path_test_images = get_gcd_paths('/data/mandonaire','test')

test_dataset = GCD(path_test_images, resize=224)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=8,
        num_workers=8,
        shuffle=True,
    )

In [14]:
len(test_dataset)

9000

## Modelo

In [15]:
from ViG.src.models import IsotropicVIG

In [16]:
model = IsotropicVIG(in_channels=192, out_channels=7, num_blocks=12, kernel_size=16, patch_size=4)

In [17]:
from torchsummary import summary

### Train con todo el batch

In [18]:
device = 'cuda:3'
EPOCHS = 200
LR = 3e-5

In [19]:
model = model.to(device)

In [20]:
from sklearn.metrics import accuracy_score

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
#optimizer = torch.optim.AdamW(model.parameters(), lr=LR)


train_losses = []
test_losses = []


for e in range(EPOCHS):
    train_predictions, train_targets, train_loss = train(model, train_loader, criterion, optimizer, device)
    test_predictions, test_targets, test_loss = test(model, test_loader, criterion, device)
    
    train_acc = accuracy_score(train_targets, train_predictions)
    test_acc = accuracy_score(test_targets, test_predictions)
    
    
    
    if e%10==0:
        print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format( 
            e+1,
            train_acc,
            train_loss,
            #test_acc,
            #test_loss
        ))
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    

 70%|███████   | 880/1250 [09:51<04:08,  1.49it/s]

In [ ]:
pd.Series(train_losses).plot.line()
pd.Series(test_losses).plot.line()
plt.show()

In [ ]:
torch.save(the_model.state_dict(), "./ViG/vig_parameters.pt")